# Assessing the Impact of Product Specifications and Brand Origin on the Pricing of Mechanical Keyboards in 2025

## Problem Statement
Global supply chains have undergone significant political and economic disruption in recent years, particularly in the technology and consumer electronics industries. 

Mechanical keyboards, an essential component of modern computing and creative work, have become a notable example of how Chinese manufacturers have entered the enthusiast market with competitive alternatives.

Historically, branding and Western design heritage contributed greatly to pricing. However, with increased transparency and direct-to-consumer models from Chinese factories, this may no longer hold true.

## Goal
This project aims to use mechanical keyboard listings as a case study to explore whether technical specifications and country/brand of origin still meaningfully influence pricing in 2025.

## Hypothesis
H₀ (Null Hypothesis): Product specifications and brand origin (e.g., Chinese vs Western brands) have no significant effect on price.

H₁ (Alternative Hypothesis): Product specifications and brand origin significantly affect price.

## Objectives
- Determine which features (e.g., switch type, brand, layout, connectivity) influence pricing.

- Analyze whether branding and origin remain significant predictors of pricing.

- Provide insights into broader trends of consumer electronics pricing post-supply-chain globalization.



# Seeing what data we are working with

In [10]:
import pandas as pd

df = pd.read_csv('../ds_capstone_project/keebfinder_keyboards2.csv')
df.head()

,title,price,layout,mount,hall_effect,hotswap,case_material,backlight,connectivity,screen,knob
0,0.01 Z62,$59,"60%,",Plate Mount,no,no,"Alu case,",yes,"Wired,",no,no
1,0.01 Z62 Blank Blank,$59,"60%,",Plate Mount,no,no,"Alu case,",yes,"Wired,",no,no
2,80retros GB65 X Click Inc,$169,"65%,",Gasket Mount,no,yes,"Alu case,",no,"Wired,",no,no
3,80retros Pad Numpad X Click Inc,$129,NaN,Gasket Mount,no,yes,"Alu case,",no,NaN,no,no
4,8BitDo Retro,$119,"80%,",Top Mount,no,yes,"Alu case,",no,"Wireless,",no,yes


In [3]:
df.shape

(2368, 11)

In [4]:
df.isna().sum()

title               0
price               0
layout            223
mount             599
hall_effect         0
hotswap             0
case_material    1357
backlight           0
connectivity      254
screen              0
knob                0
dtype: int64

## knowing there are missing values, determine what they are and deal with them

In [12]:
df['layout'].unique()

array(['60%,', '65%,', nan, '80%,', '100%,', '75%,', '96%,', '98%,',
       '95%,', '40%,', '68%,', '60%', '64%,', '66%,', '85%,', '90%,',
       '70%,', '97%,', '80%', '40%', '65%', '50%,', '100%', '87%,', '75%',
       '78%,', '84%,'], dtype=object)

In [13]:
df['mount'].unique()

array(['Plate Mount', 'Gasket Mount', 'Top Mount', nan, 'Tray Mount',
       'Sandwich Mount', 'Bottom Mount', 'PCB Mount'], dtype=object)

In [11]:
df['case_material'].unique()

array(['Alu case,', nan, 'Alu case', 'PCB Mount'], dtype=object)

In [14]:
df['connectivity'].unique()

array(['Wired,', nan, 'Wireless,', 'Wireless', 'Wired'], dtype=object)

In [ ]:
df['layout'].fillna('Unknown', inplace=True)
df['mount'].fillna('Unknown', inplace=True)
df['case_material'].fillna('Unknown', inplace=True)
df['connectivity'].fillna('Unknown', inplace=True)